In [1]:
import os
from google.colab import userdata
os.environ["HF_TOKEN"] = userdata.get('HF_TOKEN')

In [3]:
%%capture
import torch
#hides output

major_version, minor_version = torch.cuda.get_device_capability()

!pip install "unsloth[colab-new] @ git+https://github.com/unslothai/unsloth.git"


In [4]:
if major_version >= 8:
    # Use this for new GPUs like Ampere, Hopper GPUs (RTX 30xx, RTX 40xx, A100, H100, L40)

    !pip install --no-deps packaging ninja einops flash-attn xformers trl peft accelerate bitsandbytes
    print("used A100 GPU")
else:
    # Use this for older GPUs (V100, Tesla T4, RTX 20xx)

    !pip install --no-deps xformers trl peft accelerate bitsandbytes
    print("used t4 GPU")

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 218.2/218.2 MB 4.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 155.3/155.3 kB 17.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 190.9/190.9 kB 20.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 290.1/290.1 kB 20.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 102.2/102.2 MB 7.4 MB/s eta 0:00:00
used t4 GPU


In [ ]:
from unsloth import FastLanguageModel

max_seq_length = 2048 # Choose any! We auto support RoPE Scaling internally!
dtype = None # None for auto detection. Float16 for Tesla T4, V100, Bfloat16 for Ampere+
load_in_4bit = True

model, tokenizer = FastLanguageModel.from_pretrained(
        model_name = "mirajbhandari/mistral-7b-chat-finetune", # YOUR MODEL YOU USED FOR TRAINING
        max_seq_length = max_seq_length,
        dtype = dtype,
        load_in_4bit = load_in_4bit ,
    )
FastLanguageModel.for_inference(model) # Enable native 2x faster inference


In [6]:
user_question = "What does techvibe produce?"

que =[{"role": "user", "content": user_question}]

# Format user input question
formatted_question=tokenizer.apply_chat_template(que, tokenize=False)

# Send formatted question to the model for prediction
device = "cuda:0"
inputs = tokenizer(formatted_question, return_tensors="pt").to(device)

outputs = model.generate(**inputs, max_new_tokens=200,use_cache = True)
predicted_output = tokenizer.decode(outputs[0], skip_special_tokens=True)

# Print predicted output
print(predicted_output)

Setting `pad_token_id` to `eos_token_id`:2 for open-end generation.


[INST] What does techvibe produce? [/INST] Techvibe produces mobile devices such as smartphones and tablets.
